<a href="https://colab.research.google.com/github/mvince33/Coding-Dojo/blob/main/week11/mock_belt_exam3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Keras

# Sci-kit Learn

In [2]:
# Load the data
url = ''